# Run a single test

In [3]:
import os, pprint

In [4]:
EXAMPLES_FOLDER = "../examples"

In [5]:
[f for f in os.listdir(EXAMPLES_FOLDER) if f not in [".DS_Store", ".ipynb_checkpoints"]]

['.tools',
 'change_case',
 'multivariate',
 'results',
 'sacurine',
 'workflow-test-suite-2.yml',
 'workflow-test-suite-1.yml',
 'workflows.json']

In [6]:
suite_conf_filename = os.path.join(EXAMPLES_FOLDER, "workflow-test-suite-2.yml")
suite_conf_filename = os.path.join(EXAMPLES_FOLDER, "workflow-test-suite-1.yml")

In [7]:
import ruamel.yaml as yaml
with open(suite_conf_filename, "r") as fp:
    data = yaml.load(fp)
    print yaml.round_trip_dump(data, indent=2, block_seq_indent=3)

enable_logger: false
workflows:
  change_case:
    expected:
      output2: expected_output_2
      output1: expected_output_1
    inputs:
      Input Dataset: input
    base_path: change_case
    file: workflow.ga



In [8]:
from wft4galaxy import WorkflowTestConfiguration

In [9]:
help(WorkflowTestConfiguration.load)

Help on function load in module wft4galaxy:

load(filename='workflows.yml', workflow_test_name=None, file_format='YAML', output_folder=None)
    Load the configuration of a workflow test suite or a single workflow test
    from a YAML or JSON configuration file.
    
    :type filename: str
    :param filename: the path of the file containing the suite definition
    
    :type workflow_test_name: str
    :param workflow_test_name: the optional name of a workflow test
    
    :type file_format: str
    :param file_format: ``YAML`` or ``JSON``
    
    :rtype: dict or :class:`WorkflowTestConfiguration`
    :return: a dictionary containing the configuration of a workflow test suite or
        a single workflow test configuration (i.e., an instance of :class:`WorkflowTestConfiguration`)



In [10]:
test_conf = WorkflowTestConfiguration.load(suite_conf_filename, "change_case")

In [11]:
print test_conf

WorkflowTestConfig: name=change_case, file=workflow.ga, inputs=[Input Dataset], expected_outputs=[output2,output1]


In [12]:
from wft4galaxy import WorkflowTestRunner

In [13]:
help(WorkflowTestRunner.new_instance)

Help on function new_instance in module wft4galaxy:

new_instance(workflow_test_config, galaxy_url=None, galaxy_api_key=None)
    Factory method to create and initialize a new :class:`WorkflowTestRunner` instance.
    
    :type workflow_test_config: :class:`WorkflowTestConfiguration`
    :param workflow_test_config: the configuration of a workflow test
    
    :type galaxy_url: str
    :param galaxy_url: url of your Galaxy server instance.  If ``none``, the environment variable
        ``BIOBLEND_GALAXY_URL`` is used. An error is raised when such a variable cannot be found.
    
    :type galaxy_api_key: str
    :param galaxy_api_key: an API key from your Galaxy server instance.  If ``none``, the environment variable
        ``BIOBLEND_GALAXY_API_KEY`` is used. An error is raised when such a variable cannot be found.
    
    :rtype: :class:`WorkflowTestRunner`
    :return: a :class:`WorkflowTestRunner` instance



In [14]:
test_runner = WorkflowTestRunner.new_instance(test_conf)

In [15]:
help(test_runner.run_test)

Help on method run_test in module wft4galaxy:

run_test(self, base_path=None, inputs=None, params=None, expected_outputs=None, output_folder=None, disable_assertions=None, disable_cleanup=None, enable_logger=None, enable_debug=None) method of wft4galaxy.WorkflowTestRunner instance
    Run the workflow test which this runner is associated to.
    The parameters ``base_path``, ``inputs``, ``outputs``, ``expected_outputs``
    ``output_folder``, ``disable_assertions``, ``disable_cleanup``, ``enable_logger``, ``enable_debug``
    can be provided to override the corresponding defined in the :class:`WorkflowTestConfiguration` instance
    which this runner is related to (see :class:`WorkflowTestConfiguration` for more details).
    
    :rtype: :class:`WorkflowTestResult`
    :return: the :class:`WorkflowTestResult` instance which represents the test result



In [16]:
test_result = test_runner.run_test(enable_debug=True)

2016-11-09 05:44:57,762 DEBUG: Using default params
2016-11-09 05:44:57,787 DEBUG: Checking required tools ...
2016-11-09 05:44:57,897 DEBUG: Missing tools: None
2016-11-09 05:44:57,898 DEBUG: Checking required tools: DONE
2016-11-09 05:44:58,309 INFO: Create a history '_WorkflowTestHistory_441e6517-a637-11e6-b006-a8667f33b7fc' (id: u'fa6833a4eadf9064')
2016-11-09 05:44:59,506 INFO: Workflow '_WorkflowTest_TestWorkflow (imported from API)' (id: b701da857886499b) running ...
2016-11-09 05:45:16,084 INFO: Workflow '_WorkflowTest_TestWorkflow (imported from API)' (id: b701da857886499b) executed
2016-11-09 05:45:16,085 INFO: Checking test output: ...
2016-11-09 05:45:16,086 DEBUG: Checking OUTPUT 'output1' ...
2016-11-09 05:45:16,200 DEBUG: Downloaded output output1: dataset_id 'c4fa4dcd0ef9867a', filename 'results/change_case/output1'
2016-11-09 05:45:16,201 DEBUG: Configured comparator function: None
2016-11-09 05:45:16,203 DEBUG: Using default comparator....
2016-11-09 05:45:16,221 DEBU

In [17]:
print("Test %s:\n\t - workflow: [%s] \n\t - results: %r" % \
      (test_result.test_id, test_result.workflow.name, test_result.results))

Test 441e6517-a637-11e6-b006-a8667f33b7fc:
	 - workflow: [_WorkflowTest_TestWorkflow (imported from API)] 
	 - results: {u'output2': True, u'output1': True}


In [18]:
print "Outputs: ", test_result.results.keys()

Outputs:  [u'output2', u'output1']


In [19]:
test_result.passed(), test_result.failed()

(True, False)

In [20]:
test_result.check_output("output1")

True